In [158]:
#Dependencies
import pandas as pd
import numpy as np
import nltk
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Embedding, GlobalMaxPool1D, Bidirectional
from keras.layers import Dense, LSTM, Dropout, BatchNormalization, Activation
from keras.models import Sequential

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jharmse/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## TF-IDF - Akshi

## Naive Bayes - Xinbin

## Logistic Regression - Akshi

## Word2vec - Xinbin

## Multilayer Perceptron - Matt

### Data Import

For this project, we are using Kaggle's toxic comment datasets. The data, and an overview of the data, can be found [here](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data).

There are two files worth taking note of:

* `train.csv`
* `test.csv`

We will *train* our models on the `train.csv` data. To ensure that our model isn't memorising the training data (a.k.a. overfitting), we will *test* our model on the independent `test.csv` data.

`test.csv` has the same format as `train.csv`, but contains never-seen-before comment. By testing our model on this dataset, will give us an indication of whether our model will work in a real-world application (will it be able to flag or delete new toxic comments?).

In [147]:
train = pd.read_csv('../additional/data/train.csv')
test_X = pd.read_csv('../additional/data/test.csv')
test_labels = pd.read_csv('../additional/data/test_labels.csv')

print("Training data examples:")
display(train.head())
print("Test input examples:")
display(test_X.head())
print("Test expected output examples:")
display(test_labels.head())

Training data examples:


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


Test input examples:


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


Test expected output examples:


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


The test data contains a bunch of `-1` values. These should be omitted from scoring (see Kaggle link).

In [148]:
# cleaner ways of doing this, but whatevs
remove_rows = test_labels.toxic != -1
test_labels = test_labels[remove_rows]
test_X = test_X[remove_rows]

display(test_X.head())
test_labels.head()

,id,comment_text
5,0001ea8717f6de06,Thank you for understanding. I think very high...
7,000247e83dcc1211,:Dear god this site is horrible.
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig..."
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ..."
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l..."


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0


For our needs, we are only interested in wheter a comment is toxic or not. We aren't interested in the type of toxiciity. Let's change this to a binary response variable.

In [149]:
# train data
train_y = train.iloc[:, 2:] != np.zeros((len(train), 6))
train_y = train_y.any(axis=1)

# test data
test_y = test_labels.iloc[:, 1:] != np.zeros((len(test_labels), 6))
test_y = test_y.any(axis=1)

print('Training data Binary Response Variable')
print(train_y.head())
print('\nTest data Binary Response Variable')
print(test_y.head())

Training data Binary Response Variable
0    False
1    False
2    False
3    False
4    False
dtype: bool

Test data Binary Response Variable
5     False
7     False
11    False
13    False
14    False
dtype: bool


In this case, `True` represents a *toxic* comment. `False` represents *non-toxic*. However, for our mathematical models to work, we need numbers. We will convert `True` to `1` and `False` to `0`.

In [150]:
train_y = train_y.astype(float)
test_y = test_y.astype(float)
print('Training Data Response Variable')
print(train_y.head())
print('\nTest Data Response Variable')
print(test_y.head())

Training Data Response Variable
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

Test Data Response Variable
5     0.0
7     0.0
11    0.0
13    0.0
14    0.0
dtype: float64


In [151]:
train_toxic = train_y[train_y == 1.0]
train_non_toxic = train_y[train_y == 0.0]
train_non_toxic = train_non_toxic[:len(train_toxic)]

rows_keep = list(train_toxic.index)
rows_keep = keep + list(train_non_toxic)
rows_keep.sort()

train_y = train_y.iloc[rows_keep]
train = train.iloc[rows_keep]

In [152]:
train.shape

(48675, 8)

### Data Cleaning

In [153]:
def preproc_line(line):
    text = re.sub(r"[^a-zA-Z0-9]", " ", line.lower())
    words = text.split()
    words = [w for w in words if w not in stopwords.words('english')]
  
    return words

In [154]:
X = []

for line in train['comment_text']:
    X.append(preproc_line(line))

In [156]:
X[0]

['explanation',
 'edits',
 'made',
 'username',
 'hardcore',
 'metallica',
 'fan',
 'reverted',
 'vandalisms',
 'closure',
 'gas',
 'voted',
 'new',
 'york',
 'dolls',
 'fac',
 'please',
 'remove',
 'template',
 'talk',
 'page',
 'since',
 'retired',
 '89',
 '205',
 '38',
 '27']

In [167]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
X_tokenized = tokenizer.texts_to_sequences(X)

In [168]:
max_len = 200
X_pad = pad_sequences(X_tokenized, maxlen=max_len)

In [170]:
X_pad[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 14,  4,  5, 15, 21, 23,  9, 10, 24, 26, 20, 22,  6, 12,
       27, 25,  3,  8, 11,  2,  1,  7, 19, 16, 18, 17, 13], dtype=int32)

In [171]:
# number of predictors
input_dim = len(X_pad[0])

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=input_dim))
# model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid'))
# model.add(Activation('sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 32)                6432      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 6,465
Trainable params: 6,465
Non-trainable params: 0
_________________________________________________________________
None


In [172]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
             metrics=['accuracy'])

In [175]:
model.fit(X_pad, train_y, epochs=4, batch_size=32)

Epoch 1/4
48675/48675 [==============================] - 2s 48us/step - loss: 0.1115 - acc: 0.9930
Epoch 2/4
48675/48675 [==============================] - 2s 49us/step - loss: 0.0546 - acc: 0.9966
Epoch 3/4
48675/48675 [==============================] - 2s 48us/step - loss: 0.0434 - acc: 0.9973
Epoch 4/4
48675/48675 [==============================] - 2s 48us/step - loss: 0.0414 - acc: 0.9974


## Bidirectional Recurrent Neural Network(RNN) - Matt

In [181]:
def create_network(input_dim, embed_size, units, layers, output_dim, prob=0.2):
    input_ = Input(name='input_', shape=(input_dim, ))
    embed = Embedding(input_dim, embed_size)(input_)

    def add_layer(input_layer, units, name):
        lstm = Bidirectional(LSTM(units, return_sequences=True, activation='relu',
                                 name=name))(input_layer)
        bn_layer = BatchNormalization()(lstm)
        return bn_layer

    for i in range(layers):
        if i == 0:
            last_layer = add_layer(embed, units, 'rnn0')
        else:
            last_layer = add_layer(last_layer, units, 'rnn'+str(i))

    x = Dropout(prob)(last_layer)
    x = GlobalMaxPool1D()(x)
    x = Dense(units, activation='relu')(x)
    x = Dropout(prob)(x)
    logits = Dense(output_dim, name='logits')(x)
    out = Activation('sigmoid', name='out')(logits)
    model = Model(inputs=input_, outputs=out)

    print(model.summary())

    return model

In [182]:
embed_size = 128
units = 128
layers = 2
batch_size = 32
epochs = 4

In [183]:
K.clear_session()
model = create_network(max_len, embed_size, units, layers, 1)
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 128)          25600     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 256)          263168    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 256)          1024      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200, 256)          394240    
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 256)          1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 256)          0         
__________

In [184]:
model.fit(X_pad, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 43807 samples, validate on 4868 samples
Epoch 1/4
 3136/43807 [=>............................] - ETA: 15:57 - loss: 0.6160 - acc: 0.7369

KeyboardInterrupt: 

## Accuracy - Akshi

## AUC (ROC) - Akshi